<a href="https://colab.research.google.com/github/VanShi87/ML-Stepik/blob/master/Simpson_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from torchvision import datasets,models,transforms
from torch.utils.data import DataLoader
import torch
import numpy as np
from torch import nn, optim

In [0]:
#@title



In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!unzip '/content/drive/My Drive/journey-springfield.zip' 

Streaming output truncated to the last 5000 lines.
  inflating: train/simpsons_dataset/moe_szyslak/pic_1026.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1027.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1028.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1029.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1030.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1031.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1032.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1033.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1034.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1035.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1036.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1037.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1038.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1039.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1040.jpg  
  in

In [0]:
!rm -rf /content/testset/validset
!mkdir /content/testset/validset
!cp  /content/train/simpsons_dataset/* -r /content/testset/validset/ 


In [0]:
!rm /content/testset/validset/*/*[!27].jpg 
!rm /content/train/simpsons_dataset/*/*[27].jpg

In [0]:
# Image transformations
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    # Validation does not use augmentation
    'valid':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [0]:
traindir = '/content/train/simpsons_dataset/'
validdir = '/content/testset/validset/'
batch_size=32

In [0]:
# Datasets from folders
data = {
    'train':
    datasets.ImageFolder(root=traindir, transform=image_transforms['train']),
    'valid':
    datasets.ImageFolder(root=validdir, transform=image_transforms['valid']),
}

# Dataloader iterators, make sure to shuffle
dataloaders = {
    'train': DataLoader(data['train'], batch_size=batch_size, shuffle=True),
    'val': DataLoader(data['valid'], batch_size=batch_size, shuffle=True)
}


In [10]:
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [0]:
n_inputs = 2048
n_classes = 42
n_epochs = 100

In [0]:
model.fc = nn.Sequential(
                      nn.Linear(n_inputs, 1024), 
                      nn.ReLU(), 
                      nn.Dropout(0.5),
                      nn.Linear(1024, 256),
                      nn.ReLU(),
                      nn.Dropout(0.5),
                      nn.Linear(256, n_classes),                   
                      nn.LogSoftmax(dim=1))
nn.init.kaiming_normal_(model.fc[0].weight)
nn.init.kaiming_normal_(model.fc[3].weight)
nn.init.kaiming_normal_(model.fc[6].weight)
model = model.to('cuda')
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr =1e-3)

In [13]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [14]:
n_epochs_stop = 5
checkpoint_path = '/content/model.pth'
min_val_loss = np.Inf
epochs_no_improve = 0

# Main loop
for epoch in range(1,n_epochs+1):
  # Initialize validation loss for epoch
  epoch_loss = 0.0
  val_loss = 0.0
  model.train()
    
  # Training loop
  for data, targets in dataloaders['train']:
    data,targets = data.to('cuda'), targets.to('cuda')
    # Generate predictions
    out = model(data)
    optimizer.zero_grad()
    # Calculate loss
    loss = criterion(out, targets)
    # Backpropagation
    loss.backward()
    # Update model parameters
    optimizer.step()
    epoch_loss += loss.item()/len(data)

  model.eval() 
  # Validation loop
  for data, targets in dataloaders['val']:
    # Generate predictions 
    data,targets = data.to('cuda'), targets.to('cuda')
    out = model(data)
    # Calculate loss
    loss = criterion(out, targets)
    val_loss += loss.item()/len(data)

  # If the validation loss is at a minimum
  if val_loss < min_val_loss:
    # Save the model
    torch.save(model, checkpoint_path)
    epochs_no_improve = 0
    min_val_loss = val_loss
    
  else:
    epochs_no_improve += 1
    # Check early stopping condition
    if epochs_no_improve == n_epochs_stop:
      print('Early stopping!')
      break
      
  print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, epoch_loss, val_loss))
  

Epoch: 1, Training Loss: 49.63381874809662, Validation Loss: 9.653531523495912
Epoch: 2, Training Loss: 40.18301118289431, Validation Loss: 8.236408865749835
Epoch: 3, Training Loss: 37.19987681259712, Validation Loss: 7.3397164680063725
Epoch: 4, Training Loss: 35.309605891505875, Validation Loss: 7.077049298435449
Epoch: 5, Training Loss: 34.22890067100525, Validation Loss: 6.9322904418408875
Epoch: 6, Training Loss: 33.48929571236173, Validation Loss: 6.866625155210495


KeyboardInterrupt: ignored

In [0]:
!pip install scikit-image

In [15]:
import os
import pandas as pd
from PIL import Image
class_names = sorted(os.listdir('/content/train/simpsons_dataset/'))


def predict(model, test_image_name):
    test_image = Image.open('/content/testset/testset/'+test_image_name)
    transform = image_transforms['valid']  
    test_image_tensor = transform(test_image)
    test_image_tensor = test_image_tensor.view(1, 3, 224, 224).cuda()
    with torch.no_grad():
        model.eval()
        out = model(test_image_tensor)
        ps = torch.exp(out)
        topk, topclass = ps.topk(1, dim=1)
    return class_names[topclass.cpu().numpy()[0][0]]

model=torch.load('/content/model.pth')
files = [f for f in os.listdir('/content/testset/testset/')]
results = {}
for file in files:
  results[file] = predict(model,file)
df = pd.read_csv('/content/sample_submission.csv')  
for idx, row in df.iterrows():
  row[1] = results[row[0]]
print(df.head())
  



         Id             Expected
0  img0.jpg  milhouse_van_houten
1  img1.jpg         lisa_simpson
2  img2.jpg    principal_skinner
3  img3.jpg         ned_flanders
4  img4.jpg         lisa_simpson


In [0]:
df.to_csv('/content/answer.csv', index=False)

In [16]:
print(df.loc[500:550,:])

             Id                  Expected
500  img500.jpg    abraham_grampa_simpson
501  img501.jpg          krusty_the_clown
502  img502.jpg       milhouse_van_houten
503  img503.jpg          krusty_the_clown
504  img504.jpg              bart_simpson
505  img505.jpg              lisa_simpson
506  img506.jpg              ned_flanders
507  img507.jpg             homer_simpson
508  img508.jpg    apu_nahasapeemapetilon
509  img509.jpg             kent_brockman
510  img510.jpg              ned_flanders
511  img511.jpg             marge_simpson
512  img512.jpg  charles_montgomery_burns
513  img513.jpg              lisa_simpson
514  img514.jpg              chief_wiggum
515  img515.jpg              lisa_simpson
516  img516.jpg       milhouse_van_houten
517  img517.jpg          krusty_the_clown
518  img518.jpg              chief_wiggum
519  img519.jpg       milhouse_van_houten
520  img520.jpg             homer_simpson
521  img521.jpg               moe_szyslak
522  img522.jpg              lisa_